通过实例研究清楚深度学习究竟干了什么
===
我们前面介绍了卷积神经网络中常用的一些层，现在我们通过一个实例来把他们都串起来。我们使用的数据集是mnist，具体的网络结构如下

In [1]:
from JLib.pyn.layers import Conv2D, Relu, MaxPooling, FullyConnect, Softmax
batch_size = 64
conv1 = Conv2D([batch_size, 28, 28, 1], 12, 5, 1)
relu1 = Relu(conv1.output_shape)
pool1 = MaxPooling(relu1.output_shape)
conv2 = Conv2D(pool1.output_shape, 24, 3, 1)
relu2 = Relu(conv2.output_shape)
pool2 = MaxPooling(relu2.output_shape)
fc = FullyConnect(pool2.output_shape, 10)
sf = Softmax(fc.output_shape)

# 1.前向传播
我们使用的是mnist数据集，img的尺寸是$[64, 28, 28, 1]$

In [ ]:
conv1_out = conv1.forward(img)
relu1_out = relu1.forward(conv1_out)
pool1_out = pool1.forward(relu1_out)
conv2_out = conv2.forward(pool1_out)
relu2_out = relu2.forward(conv2_out)
pool2_out = pool2.forward(relu2_out)
fc_out = fc.forward(pool2_out)
batch_loss += sf.cal_loss(fc_out, np.array(label))
train_loss += sf.cal_loss(fc_out, np.array(label))

## 1.1.Conv2D

卷积层的前向计算就是进行卷积计算的过程，我们知道下一个神经元的输出就是
$$\phi_i=\omega_i \bullet x + b$$
对于整个特征图来说，就是
$$\vec{\Phi}=\vec{\omega} \bullet \vec{x}+\vec{b}$$
假设我们的输入特征图为$bs,224,224, 3$(bs为图片张数,224为尺寸大小,3为通道数-RGB)，输出神经元output_channel=64，卷积核大小ksize=3，步长strid=1，补齐，name最终输出的结果就应该是$bs,224,224,64$

对于一个batch中的任何一幅图片，我们都需要做$\phi_i=\omega_i \bullet x + b$，对于我们的例子来说，$\omega$的尺寸为$(ksize, ksize, inchannels, outchannels)$，也就是$(5, 5, 1, 12)$,偏置b的尺寸为$(12, )$。首先我们要做的是$\omega$与$x$的点乘，直接用输入的图片大小做点乘是不可以的，我们需要对当前的图片做一个转换，也就是_im2col函数

In [ ]:
def _im2col(image, ksize, stride):
    # image is a 4d tensor([batchsize, width ,height, channel])
    image_col = []
    for i in range(0, image.shape[1] - ksize + 1, stride):
        for j in range(0, image.shape[2] - ksize + 1, stride):
            col = image[:, i:i + ksize, j:j + ksize, :].reshape([-1])
            image_col.append(col)
    image_col = np.array(image_col)

    return image_col

这个函数是这样的，从图片的左上角开始，以$(ksize, ksize)$的尺寸做滑动窗口，总共可以得到$24 \times 24 = 576$个卷积核，每个卷积核拉平到一维向量，然后将576个合在一起，就成了$(576, 25)$

接下来用这个数据与weights做点乘，然后加上偏置，就是卷积的结果。即

In [ ]:
np.dot(self.image, self.weights) + self.bias

这里需要把weights的shape变为$25, 12$，才可以和$576, 25$做点乘，然后加上$12,$，结果的尺寸就是$576, 12$，然后将这个结果reshape成$(1, 24, 24, 12)$。整个batch的结果就是$(batch, 24, 24, 12)$

## 1.2.ReLU

RuLU的逻辑很简单，对于Conv2D过来的结果，如果小于0，就用0替换，如果大于0，那就保持不变，即

In [ ]:
np.maximum(x, 0)

结果的shape还是$(bs, 24, 24, 12)$

## 1.3.Pool

### 1.3.1.MaxPool-最大池化

![images](images/05_03_02.jpeg)

我们可以很明显的看到，就是从图片上按照ksize，stride来找到每一个区块，然后找到这个区块内各个值得最大值，然后放入结果中的相应位置上，index是用来后期做反向传播用的，index记录每个区块内最大值的位置

### 1.3.2.AvaragePool-平均池化

![images](images/05_03_03.jpeg)

我们可以很明显的看到，就是从图片上按照ksize，stride来找到每一个区块，然后找到这个区块内各个值的平均值，然后放入结果中的相应位置上，index是用来后期做反向传播用的。本例中，使用了stride=2，所以结果的shape就是$(bs,12, 12, 12)$

接着又进行了一个Conv2D，ReLU，MaxPool，结果出来的shape是$(bs, 5, 5, 24)$

## 1.4.FullConnect

将上述结果reshape成$bs, 5 \times 5 \times 24=600$,weights的shape就是$600, 10$，10就是mnist的分类数目，做完点乘之后，然后加上bias，就是最后的结果

前向传播的结果就是上述的结果，shape=$64, 10$

# 2.计算损失

我们使用的是Softmax损失，传入前向传播的结果，以及这一个batch对应的mnist的label

![images](images/05_09_01.jpeg)

上图是这么计算的。首先计算$e^3 \approx 20$, $e^1 \approx 2.7$, $e^{-3} \approx 0.05$,然后计算他们三个的和$20+2.7+0.05=22.75$。接着开始计算$y_1=\frac{20}{22.75} \approx 0.88$，$y_2=\frac{2.7}{22.75} \approx 0.12$，$y_3=\frac{0.05}{22.75} \approx 0$

在真正的代码里，会分为两步计算，首先计算softmax，然后在计算loss

## 2.1.计算softmax

In [1]:
def predict(self, prediction):
    exp_prediction = np.zeros(prediction.shape)
    self.softmax = np.zeros(prediction.shape)
    for i in range(self.batchsize):
        prediction[i, :] -= np.max(prediction[i, :])
        exp_prediction[i] = np.exp(prediction[i])
        self.softmax[i] = exp_prediction[i]/np.sum(exp_prediction[i])
    return self.softmax

softmax的shape是$64,10$,循环每个图片的前向传播结果。做三件事
- 首先要减去最大值。
- 计算$e^x$。
- 计算当前结果的softmax,$\frac{e^x}{\sum e^i}$

## 2.2.计算loss

In [ ]:
for i in range(self.batchsize):
    self.loss += np.log(np.sum(np.exp(prediction[i]))) - prediction[i, label[i]]

计算loss用的是最大似然的原理，其实也就是交叉熵损失

# 3.梯度计算与反向传播

In [ ]:
eta = sf.gradient()
eta = fc.backward(sf.eta, alpha=learning_rate, weight_decay=0.0004)
eta = pool2.backward(eta, alpha=learning_rate, weight_decay=0.0004)
eta = relu2.backward(eta, alpha=learning_rate, weight_decay=0.0004)
eta = conv2.backward(eta, alpha=learning_rate, weight_decay=0.0004)
eta = pool1.backward(eta, alpha=learning_rate, weight_decay=0.0004)
eta = relu1.backward(eta, alpha=learning_rate, weight_decay=0.0004)
eta = conv1.backward(eta, alpha=learning_rate, weight_decay=0.0004)

## 3.1.Softmax

In [ ]:
def gradient(self):
    self.eta = self.softmax.copy()
    for i in range(self.batchsize):
        self.eta[i, self.label[i]] -= 1
    return self.eta

损失函数就是反向传播的起点，由于没有weights和bias，那么需要计算的就是$\frac{\partial{loss}}{\partial{z}}$

$$
loss=-loglikelyhood=log(\sum_{j=1}^me^{z_j})-z_y
$$

$$
\frac{\partial{loss}}{\partial{z_k}}=\frac{e^{z_k}}{\sum_{j=1}^me^{z_j}}-\delta_{ky}=\sigma_k(z)-\delta_{xy}, \delta_{ky}=\begin{cases}
0 & y \neq k \\
1 & y=k
\end{cases}
$$

self.eta的shape大小与softmax一致，都是$64, 10$，只要将$y=k$的值减去1就行了。

## 3.2.FullConnect

我们知道，全连接层的前向传播公式为$y=w \bullet x + b$，那么就有

$$
\begin{cases}
\delta_x=\frac{\partial{Loss}}{\partial{x}}=\frac{\partial{Loss}}{\partial{y}} \bullet w^T=\delta_{x+1} \bullet w^T \\
\frac{\partial{Loss}}{\partial{w}}=\frac{\partial{Loss}}{\partial{y}} \bullet \frac{\partial{y}}{\partial{w}}=x \bullet \delta_{x+1}^T \\
\end{cases}
$$

$\delta_{x+1}$就是传入的eta。eta的shape是$64,10$,weights的shape是$600, 10$

In [ ]:
next_eta = np.dot(eta, self.weights.T) #shape:64,600
next_eta = np.reshape(next_eta, self.input_shape) # shape: 64, 5, 5, 24

for i in range(eta.shape[0]):
    col_x = self.x[i][:, np.newaxis]
    eta_i = eta[i][:, np.newaxis].T
    self.w_gradient += np.dot(col_x, eta_i)
    self.b_gradient += eta_i.reshape(self.bias.shape)
self.weights *= (1 - weight_decay)
self.bias *= (1 - weight_decay)
self.weights -= alpha * self.w_gradient
self.bias -= alpha * self.bias
# zero gradient
self.w_gradient = np.zeros(self.weights.shape)
self.b_gradient = np.zeros(self.bias.shape)

next_eta是继续往前传递的梯度。

## 3.3.MaxPooling

## 3.4.ReLU